# Differential Diffusion with Python

In [ ]:
!pip install accelerate diffusers

In [ ]:
import numpy as np
from PIL import Image
import torch

from torchvision import transforms
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import make_image_grid

#Load input image

In [ ]:
input_image = Image.open("landscape-1.jpg")

In [ ]:
input_image.size

In [ ]:
height = 876
width = 1280

# Create a numpy array for the image
gradient = np.zeros((height, width), dtype=np.uint8)

# Generate the gradient
for y in range(height):
    color_value = int(255 * y / (height))
    gradient[y, :] = color_value

# Convert the numpy array into an image
mask = Image.fromarray(gradient, mode='L')

# flip horizontal
mask = mask.transpose(Image.FLIP_TOP_BOTTOM)

In [ ]:
make_image_grid([input_image, mask], cols=2, rows=1)

In [ ]:
def preprocess_image(image):
    image = image.convert("RGB")
    image = transforms.CenterCrop((image.size[1] // 64 * 64, image.size[0] // 64 * 64))(image)
    image = transforms.ToTensor()(image)
    image = image * 2 - 1
    image = image.unsqueeze(0).to("cuda")
    return image

def preprocess_map(map):
    map = map.convert("L")
    map = transforms.CenterCrop((map.size[1] // 64 * 64, map.size[0] // 64 * 64))(map)
    map = transforms.ToTensor()(map)
    map = map.to("cuda")
    return map

In [ ]:
image = preprocess_image(input_image)
mask = preprocess_map(mask)

In [ ]:
pipe = DiffusionPipeline.from_pretrained(
    "SG161222/RealVisXL_V4.0",
    torch_dtype=torch.float16,
    variant="fp16",
    custom_pipeline="pipeline_stable_diffusion_xl_differential_img2img"
)

pipe.enable_model_cpu_offload()

In [ ]:
pipe.scheduler = DPMSolverMultistepScheduler.from_config(
    pipe.scheduler.config,
    use_karras_sigmas=True
)

In [ ]:
prompt = """
(masterpiece:1.2), a rainy mushroom garden, unreal engine cinematic smooth,
intricate detail, kids story book style, muted colors, watercolor,
8k, highly detailed, (best quality:1.2)
"""

negative_prompt = "blurry, low quality, lowres"

output_image = pipe(

    prompt              = prompt,
    negative_prompt     = negative_prompt,
    guidance_scale      = 7,
    num_inference_steps = 30,
    original_image      = image,
    image               = image,
    strength            = 0.9,
    map                 = mask).images[0]

In [ ]:
make_image_grid([input_image, output_image], cols=2, rows=1)